In [22]:
import pandas as pd
import json
import requests
import aiohttp
import asyncio
import numpy as np
import pendulum

from pathlib import Path

## Note:
See `240220_sl_surf_spots.ipynb` for spot getter

In [2]:
response = requests.get("https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0")

* You will not get Surfline forecast data without a valid Surfline premium login. Add your credentials to `.env.development`:
  ```
  SURFLINE_EMAIL=xxx
  SURFLINE_PASSWORD=yyy
  ```

##### Requests

`https://services.surfline.com/kbyg/spots/forecasts/{type}?{params}`


Type|Data
----|----
rating|array of human-readable and numeric (0-6) ratings
wave|array of min/max sizes & optimal scores
wind|array of wind directions/speeds & optimal scores
tides|array of types & heights
weather|array of sunrise/set times, array of temperatures/weather conditions

Param|Values|Effect
-----|------|------
spotId|string|Surfline spot id that you want data for. A typical Surfline URL is `https://www.surfline.com/surf-report/venice-breakwater/590927576a2e4300134fbed8` where `590927576a2e4300134fbed8` is the `spotId`
days|integer|Number of forecast days to get (Max 6 w/o access token, Max 17 w/ premium token)
intervalHours|integer|Minimum of 1 (hour)
maxHeights|boolean|`true` seems to remove min & optimal values from the wave data output
sds|boolean|If true, use the new LOTUS forecast engine
accesstoken|string|Auth token to get premium data access (optional)

Anywhere there is an `optimalScore` the value can be interpreted as follows:

Value|Meaning
-----|-------
0|Suboptimal
1|Good
2|Optimal


In [3]:
types = ["rating", "wave", "wind", "tides", "weather"]
params = ["spotId", "days", "intervalHours", "maxHeights", "sds", "accesstoken"]
base = "https://services.surfline.com/kbyg/spots/forecasts"

In [4]:
datapath = Path('./data')


In [5]:
df = pd.read_csv(datapath/'spot_list.csv')


In [7]:
df.head()

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.11700,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.54900,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.56200,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.73700,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.91913,30.230299,https://www.surfline.com/surf-report/morgantow...


Get the spot `id` for 1st Street Jetty in Va Beach

In [8]:
jetty_id = df[df['names'].str.contains('1st Street Jetty', case=False, na=False)]['ids'].values[0]
jetty_id

'584204214e65fad6a7709ce7'

In [9]:
ex_params = {params[0]: jetty_id}
ex_params

{'spotId': '584204214e65fad6a7709ce7'}

Surfline seems to change their spot IDs periodically. Check a spot on the website and pass the objectId from the url as a param to debug if this is the case. If they've changed you'll need to run the notebook `240220_sl_surf_spots.ipynb` as mentioned above to refresh the spots dataset

In [ ]:
debug_params = {params[0]: "584204214e65fad6a7709ce7"}

In [16]:
res = requests.get(f"{base}/{types[0]}", params=ex_params)
res.status_code

200

In [17]:
rating_json = res.json()
rating_json

{'associated': {'location': {'lon': -75.96648, 'lat': 36.83036135089083},
  'runInitializationTimestamp': 1711670400},
 'data': {'rating': [{'timestamp': 1711684800,
    'utcOffset': -4,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1711688400,
    'utcOffset': -4,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 1.83}},
   {'timestamp': 1711692000,
    'utcOffset': -4,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 1.67}},
   {'timestamp': 1711695600,
    'utcOffset': -4,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 1.5}},
   {'timestamp': 1711699200,
    'utcOffset': -4,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 1.5}},
   {'timestamp': 1711702800,
    'utcOffset': -4,
    'rating': {'key': 'POOR', 'value': 1.17}},
   {'timestamp': 1711706400,
    'utcOffset': -4,
    'rating': {'key': 'POOR', 'value': 1}},
   {'timestamp': 1711710000,
    'utcOffset': -4,
    'rating': {'key': 'POOR', 'value': 1.17}},
   {'timestamp': 1711713600,
    'utcOffset': -4,
    '

Convert a unix timestamp -> utc

In [29]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 3, 29, 4, 0, 0, tzinfo=Timezone('UTC'))

The `utcOffset` field seems to be aware that I'm working in EST currently. Either that or it's the time coding for the spot itself

***

### Ratings